In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random

In [4]:
!kaggle competitions download -c titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile

zp = zipfile.ZipFile('titanic.zip') 
df = pd.read_csv(zp.open('train.csv'))

In [6]:
df = df.drop(['PassengerId', 'Name', 'Parch', 'Ticket', 'Cabin'], axis = 1)

df.loc[df['SibSp'] >= 1, 'SibSp'] = 1
df['Embarked'] = df['Embarked'].fillna('S')

df = df.dropna(subset = ['Age'])
df['Age'] = df['Age'].astype(int)

df.head()

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,male,22,1,7.2500,S
1,1,1,female,38,1,71.2833,C
2,1,3,female,26,0,7.9250,S
3,1,1,female,35,1,53.1000,S
4,0,3,male,35,0,8.0500,S


In [7]:
df = pd.concat([df, pd.get_dummies(df['Sex'], prefix='Sex')],axis=1)
df = pd.concat([df, pd.get_dummies(df['Pclass'], prefix='Pclass')],axis=1)
df = pd.concat([df, pd.get_dummies(df['Embarked'], prefix='Embarked')],axis=1)
  
df = df.drop(['Sex', 'Pclass', 'Embarked'], axis = 1)
df.head()

,Survived,Age,SibSp,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,0,22,1,7.2500,0,1,0,0,1,0,0,1
1,1,38,1,71.2833,1,0,1,0,0,1,0,0
2,1,26,0,7.9250,1,0,0,0,1,0,0,1
3,1,35,1,53.1000,1,0,1,0,0,0,0,1
4,0,35,0,8.0500,0,1,0,0,1,0,0,1


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['Survived'], axis = 1), df['Survived'], test_size=0.2)
y_train

216    1
550    1
658    0
346    1
862    1
      ..
187    1
765    1
725    0
294    0
21     1
Name: Survived, Length: 571, dtype: int64

In [9]:
class NN:
    def __init__(self, n_hidden_units = 100, learning_rate = 0.001, activation_func = 'tanh'):
        self.learning_rate = learning_rate
        self.n_hidden_units = n_hidden_units
        
        if activation_func == 'tanh':
            self.activation_func = lambda x: np.tanh(x)
            self.activation_func_derivative = lambda x: 1 - np.tanh(x)**2
        if activation_func == 'sigm':
            self.activation_func = lambda x: 1 / (1 + np.exp(-x))
            self.activation_func_derivative = lambda x: np.exp(-x) / (1 + np.exp(-x))**2
            
            
    def fit(self, X_train, y_train):
        self.X = X_train
        self.y = y_train
        
        self.n_features = len(self.X.columns)
        
        #z1 = x * W1 + b1 (col * len x len * hd) + col * cl
        #a1 = act_func(z1) 
        #z2 = a1 * W2 + b2
        #a2 = act_func(z2)
        #y_pred = argmax(a2)
        
        self.W1 = np.random.normal(0, 0.01, (self.n_hidden_units, len(self.y)))
        self.W2 = np.random.normal(0, 0.01, (self.n_hidden_units, 1))
        self.b1 = np.zeros((self.n_hidden_units, 1))
        self.b2 = np.zeros((1, len(self.y)))
        
        for i in range(10000):
            #Forward propagation
            z1 = self.W1.dot(self.X.T) + self.b1
            a1 = self.activation_func(z1)
            z2 = self.W2.dot(a1) + self.b2
            a2 = self.activation_func(z2)
            
            print(a1)
            #Backpropagation
            
            
            self.W1 -= self.learning_rate * dW1
            self.W2 -= self.learning_rate * dW2
            self.b1 -= self.learning_rate * db1
            self.b2 -= self.learning_rate * db2
            
        def predict(self, X_test):
            return self.activation_func(self.W2.dot(self.activation_func(self.W1.dot(X_test.T) + self.b1)) + self.b2)

In [10]:
clf = NN()
print('tanh', clf.activation_func(np.array([0.2, 0.5])))
print('tanh derivative', clf.activation_func_derivative(np.array([0.2, 0.5])))
print()

clf = NN(activation_func = 'sigm')
print('sigmoid', clf.activation_func(np.array([0.2, 0.5])))
print('sigmoid derivative', clf.activation_func_derivative(np.array([0.2, 0.5])))

tanh [0.19737532 0.46211716]
tanh derivative [0.96104298 0.78644773]

sigmoid [0.549834   0.62245933]
sigmoid derivative [0.24751657 0.23500371]


In [11]:
clf.fit(X_train[:10], y_train[:10])

[[0.51759694 0.52097328 0.51780118 ... 0.51767132 0.51765596 0.51783573]
 [0.49708021 0.49896341 0.49698213 ... 0.49662461 0.49683014 0.49764887]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2, 100) (100, 2)


ValueError: operands could not be broadcast together with shapes (2,100) (100,2) (2,100) 

In [ ]:
from sklearn.neural_network import MLPClassifier

clf2 = MLPClassifier(activation = 'logistic', solver = 'sgd', learning_rate_init = 0.033)
clf2.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf2.predict(X_test))

In [189]:
clf2 = MLPClassifier()
clf2.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, clf2.predict(X_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7972027972027972

In [13]:
np.zeros((1, 10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])